In [1]:
import sys

print(sys.path)

['c:\\Users\\caudtomm\\AppData\\Local\\miniconda3\\envs\\calcium_postp\\python313.zip', 'c:\\Users\\caudtomm\\AppData\\Local\\miniconda3\\envs\\calcium_postp\\DLLs', 'c:\\Users\\caudtomm\\AppData\\Local\\miniconda3\\envs\\calcium_postp\\Lib', 'c:\\Users\\caudtomm\\AppData\\Local\\miniconda3\\envs\\calcium_postp', '', 'c:\\Users\\caudtomm\\AppData\\Local\\miniconda3\\envs\\calcium_postp\\Lib\\site-packages', 'c:\\Users\\caudtomm\\AppData\\Local\\miniconda3\\envs\\calcium_postp\\Lib\\site-packages\\win32', 'c:\\Users\\caudtomm\\AppData\\Local\\miniconda3\\envs\\calcium_postp\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\caudtomm\\AppData\\Local\\miniconda3\\envs\\calcium_postp\\Lib\\site-packages\\Pythonwin']


In [ ]:
import h5py

filename = 'W:/groups/scratch/gfriedri/processed-data/250129_TC_invivoCaIMG_odorexp004_005/experiment_PR_250519.mat'

with h5py.File(filename, 'r') as f:
    # List top-level keys
    print(list(f.keys()))
    print(f['experiment'].keys())
    print(f['experiment'])
    print(type(f['experiment']))

    # Access a specific dataset
    experiment = f['experiment']

print(type(experiment))

['#refs#', '#subsystem#', 'experiment']
<KeysViewHDF5 ['name', 'series', 'summaryTable']>
<HDF5 group "/experiment" (3 members)>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>


In [32]:
import h5py
import numpy as np

def decode_str(val):
    if isinstance(val, bytes):
        return val.decode('utf-8')
    elif isinstance(val, np.ndarray) and val.dtype.kind in {'S', 'O'}:
        try:
            return np.vectorize(lambda x: x.decode('utf-8'))(val)
        except Exception:
            return val
    return val

def read_cell_array(refs, file):
    return [h5py_to_dict(file[r]) if isinstance(file[r], h5py.Group) else file[r][()] for r in refs]

def h5py_to_dict(obj):
    if isinstance(obj, h5py.Dataset):
        data = obj[()]
        return decode_str(data)
    elif isinstance(obj, h5py.Group):
        out = {}
        for key, item in obj.items():
            if isinstance(item, h5py.Dataset):
                if item.dtype == 'object':  # likely a cell array
                    refs = item[()]
                    if refs.ndim == 0:  # single ref
                        refs = [refs.item()]
                    else:
                        refs = refs.flat
                    out[key] = read_cell_array(refs, obj.file)
                else:
                    out[key] = decode_str(item[()])
            elif isinstance(item, h5py.Group):
                out[key] = h5py_to_dict(item)
            else:
                out[key] = None
        return out
    return obj

# Usage
with h5py.File(filename, 'r') as f:
    my_struct = h5py_to_dict(f['experiment'])



In [34]:
print(my_struct['series'].keys())

AttributeError: 'list' object has no attribute 'keys'